In [2]:
from AOC_utils import get_day
import numpy as np

day = 12
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Day 12 input already downloaded


['??????#??#?? 1,1,5,1',
 '?#?#??##?#? 2,5,1',
 '????????.?#???#??##? 2,1,2,1,1,6',
 '#???#?????.?#?. 2,1,2,1',
 '?##..?#?#?? 2,4',
 '..#??##????###????#? 6,8',
 '???#..?#?? 1,1,2']

In [3]:
example = '''
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
'''.split('\n')[1:-1]

In [4]:
len(input_data)

1000

In [42]:
def parse_data(data):

    samples = [[int(x)-1 for x in list(line.split(' ')[0].replace('#', '2').replace('.', '1').replace('?', '0'))] for line in data]
    groups = [[int(x) for x in line.split(' ')[1].split(',')] for line in data]

    
    def check_sample_groups(sample, groups):
        if len(groups) == 0 and sample.count(1) == 0:
            return True
        
        if len(groups) == 0 and sample.count(1) != 0:
            # print("no groups left, but still 1s")
            return False
        elif len(sample) < (sum(groups) + (max(len(groups) - 1, 0))):
            # print("sample length doesn't match possible groups")
            return False
        elif len(sample) == (sum(groups) + (max(len(groups) - 1, 0))):
            at = 0
            for group in groups:
                if sample[at:at+group].count(0) > 0:
                    # print("sample has 0s inside groups")
                    return False
                at += group + 1

        count1s = sample.count(1)
        countunknowns = sample.count(-1)

        if count1s > sum(groups):
            # print("too many ones")
            return False
        elif count1s + countunknowns < sum(groups):
            # print("not enough possible ones")
            return False

        subsections = []
        current_subsection = []
        for i in range(len(sample)):
            if sample[i] != 0:
                current_subsection.append(sample[i])
            elif len(current_subsection) > 0 or i == len(sample)-1:
                    subsections.append(current_subsection)
                    current_subsection = []

        if len(subsections) > 0:
            if len(subsections[0]) < groups[0] and 1 in subsections[0]:
                return False

        return True
    
    global_cache = {}

    def dfs(samples_remaining, groups_left):

        #stringify inputs and check cache
        key = "".join([str(x) for x in samples_remaining]).replace('-1', '?')
        # if len(key) < 250:
        key += "|" + "".join([str(x)+" " for x in groups_left])
        if key in global_cache:
            return global_cache[key]

        is_valid = check_sample_groups(samples_remaining, groups_left)

        # print(samples_remaining, groups_left, is_valid)
        if not is_valid:
            return 0
        
        while len(samples_remaining) > 0 and samples_remaining[0] == 0:
            samples_remaining = samples_remaining[1:]

        if len(samples_remaining) == 0 and len(groups_left) == 0 or len(groups_left) == 0 and samples_remaining.count(1) == 0:
            # print("returning 1")
            return 1
        else:
            next_non_zero_subset = []
            for i in range(len(samples_remaining)):
                if samples_remaining[i] != 0:
                    next_non_zero_subset.append(i)
                else:
                    break
            if samples_remaining[0] == 1:
                if len(next_non_zero_subset) >= groups_left[0]:
                    if len(samples_remaining) >= groups_left[0]+1:
                        if samples_remaining[groups_left[0]+1-1] != 1:
                            return dfs(samples_remaining[min(groups_left[0]+1, len(samples_remaining)):], groups_left[1:])
                        else:
                            # print("can't space 0 in between groups")
                            return 0
                    else:
                        return dfs(samples_remaining[groups_left[0]:], groups_left[1:])
                else:
                    # print("next non zero subset fit with next group length")
                    return 0
            else:
                zero_case = dfs(samples_remaining[1:], groups_left)

                one_case = 0
                if len(next_non_zero_subset) >= groups_left[0]:
                    if len(samples_remaining) >= groups_left[0]+1:
                        if samples_remaining[groups_left[0]+1-1] != 1:
                            one_case = dfs(samples_remaining[groups_left[0]+1:], groups_left[1:])
                    else:
                        one_case = dfs(samples_remaining[groups_left[0]:], groups_left[1:])

                output = zero_case + one_case
                
                # save to cache
                global_cache[key] = output
                return output


    total = 0
    for sample, group in zip(samples, groups):
        total += dfs(sample, group)
    print("part 1:", total)


    total2 = 0
    for sample, group in zip(samples, groups):
        # unfold samples and groups
        sample = sample + [-1] + sample + [-1] + sample + [-1] + sample + [-1] + sample
        group = group + group + group + group + group
        
        total2 += dfs(sample, group)

    print("part 2:", total2)
    print("cache length", len(global_cache))

In [43]:
parse_data(input_data)

part 1: 6488
cache length 99708
part 2: 815364548481


In [35]:
parse_data(example)

largest sample: [-1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1] [1, 4, 1, 3, 1, 1]
largest sample check 4
cache p1 {'?|1 ': 1, '???|1 1 ': 1, '???????|3 1 1 ': 1, '??|1 ': 2, '????|1 1 ': 3, '????????|3 1 1 ': 4, '?0????????|1 3 1 1 ': 4, '??????0????????|4 1 3 1 1 ': 4, '????????0????????|1 4 1 3 1 1 ': 4}

largest sample check part2 9604


part 1: 21


part 2: 525152
